In [1]:
#generating data from directory
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
train_dir='/home/nouman/Downloads/dataDir/train'
#train_dir='/content/drive/MyDrive/dataDir/train'
test_dir='/home/nouman/Downloads/dataDir/test'
#test_dir='/content/drive/MyDrive/dataDir/test'
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [2]:
from keras import Input,Model
from keras.layers import Conv2D,MaxPooling2D,UpSampling2D,Dense,Flatten
from keras import models
import tensorflow as tf
from tensorflow.keras.regularizers import l2
input_img = Input(shape=(128, 128, 3))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
decoded= UpSampling2D((2, 2))(decoded)
#decoded=  Flatten()(decoded)
#implementing svm using dense layer
#svm = Dense(3, activation='sigmoid',kernel_regularizer=l2(0.01))(decoded)
autoencoder= Model(input_img, decoded)
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 64, 64, 64)        0     

In [3]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 128, 128, 32))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                            target_size=(128, 128),
                                            batch_size=batch_size,
                                            class_mode='binary')

    i = 0
    for inputs_batch, labels_batch in generator:
        #autoencoder.compile(optimizer='adam', metrics = ['accuracy'],
               # loss='categorical_crossentropy')
        features_batch = autoencoder.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
#Note that because generators directory, yield data indefinitely in a loop, you must break after every image 
#has been seen once
        if i * batch_size >= sample_count:
            break
    return features, labels

In [4]:
#trained features and labels using conv_base
train_features, train_labels = extract_features(train_dir, 60)
#validation features and labels using conv_base
#validation_features, validation_labels = extract_features(validation_dir, 1000)
#test features using and labels conv_base
test_features, test_labels = extract_features(test_dir, 60)

Found 60 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [5]:
train_features.shape

(60, 128, 128, 32)

In [6]:
train_features = np.reshape(train_features, (-1, 128 * 128 * 32))
test_features = np.reshape(test_features, (-1, 128 * 128 * 32))

In [7]:
from sklearn import svm
clf = svm.SVC(kernel='poly')
clf.fit(train_features, train_labels) 

SVC(kernel='poly')

In [8]:
yhat = clf.predict(test_features)
yhat [0:5]

array([0., 1., 1., 2., 2.])

In [9]:
from sklearn.metrics import f1_score
f1_score(test_labels, yhat, average='weighted')

1.0

In [10]:
from sklearn.metrics import jaccard_score
jaccard_score(test_labels, yhat,average='macro')

1.0

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(confusion_matrix(test_labels, yhat))
#np.set_printoptions(precision=2)
print (classification_report(test_labels, yhat))

[[20  0  0]
 [ 0 20  0]
 [ 0  0 20]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        20
         1.0       1.00      1.00      1.00        20
         2.0       1.00      1.00      1.00        20

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60



In [12]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_features, train_labels)
y_pred = gnb.predict(test_features)

In [13]:
from sklearn.metrics import f1_score
f1_score(test_labels, y_pred, average='weighted')

1.0

In [14]:
from sklearn.metrics import jaccard_score
jaccard_score(test_labels, y_pred,average='macro')

1.0

In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, y_pred)

1.0

In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(confusion_matrix(test_labels, y_pred))
#np.set_printoptions(precision=2)

print (classification_report(test_labels, y_pred))

[[20  0  0]
 [ 0 20  0]
 [ 0  0 20]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        20
         1.0       1.00      1.00      1.00        20
         2.0       1.00      1.00      1.00        20

    accuracy                           1.00        60
   macro avg       1.00      1.00      1.00        60
weighted avg       1.00      1.00      1.00        60

